In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import dapgen
import admix_genet_cor
import admix
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

In [2]:
ROOT_DIR = (
    "/u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr"
)
PFILE_DIR = os.path.join(ROOT_DIR, "imputed")

SUPP_TABLE_URL = "https://www.dropbox.com/s/jck2mhjby2ur55j/supp_tables.xlsx?dl=1"

trait_list = [
    f.split("/")[-1].split(".")[0]
    for f in glob.glob(os.path.join(ROOT_DIR, "pheno", "*.tsv"))
]
trait_info = pd.read_excel(SUPP_TABLE_URL, sheet_name="trait-info")
trait_list = trait_info["trait"].values
dict_trait_display_name = {
    row["trait"]: row["display-name"] for _, row in trait_info.iterrows()
}

In [3]:
#     ### include GWAS variants as covariates
#     clump_path = join(CLUMP_DIR, trait, "PLINK.imputed.clumped")
#     if (sum(1 for line in open(clump_path)) > 1) and trait != "total_wbc_cnt":
#         # when there is clumped variants
#         df_clump = pd.read_csv(
#             join(CLUMP_DIR, trait, "PLINK.imputed.clumped"), delim_whitespace=True
#         )
#         # include variants with h2 > 0.004
#         df_clump = df_clump[df_clump.P < scipy.stats.chi2.sf(dset.n_indiv * 0.004, 1)]
#         if len(df_clump) > 0:
#             print(f"{len(df_clump)} SNPs with expected h2 > 0.2%")
#             df_clump_geno = []
#             for chrom, df_chrom_clump in df_clump.groupby("CHR"):
#                 dset_chrom = admix.io.read_dataset(
#                     join(PFILE_DIR, f"chr{chrom}"),
#                     n_anc=2,
#                 )
#                 dset_chrom = dset_chrom[
#                     df_chrom_clump.SNP.values, dset.indiv.index.values
#                 ]
#                 df_clump_geno.append(
#                     pd.DataFrame(
#                         dset_chrom.geno.sum(axis=2).T.compute(),
#                         columns=dset_chrom.snp.index.values,
#                         index=dset_chrom.indiv.index.values,
#                     )
#                 )
#             df_clump_geno = pd.concat(df_clump_geno, axis=1)
#             print(df_clump_geno)
#             df_covar = pd.merge(
#                 df_covar, df_clump_geno, left_index=True, right_index=True
#             )

In [4]:
# trait_list = ["total_wbc_cnt"]
for trait in trait_list:
    df_trait = (
        pd.read_csv(os.path.join(ROOT_DIR, f"pheno/{trait}.tsv"), sep="\t", index_col=0)
        .dropna(subset=[trait])
        .rename(columns={trait: "pheno"})
    )
    df_trait.to_csv(f"out/pheno/{trait}.tsv", sep="\t")
    # trait-specific treatment
    #     if trait == "total_wbc_cnt":
    # add duffy SNP if trait is total_wbc_cnt
    dset = admix.io.read_dataset(
        os.path.join(ROOT_DIR, "imputed", "chr1"),
        n_anc=2,
    )[:, df_trait.index.values]
    # find closest SNPs
    duffy_snp_loc = np.argmin(np.abs(dset.snp.POS - 159204893))
    assert dset.snp.CHROM.iloc[duffy_snp_loc] == 1
    duffy_lanc = dset[duffy_snp_loc].lanc.sum(axis=[0, 2]).compute()
    df_trait["duffy_lanc"] = duffy_lanc
    df_trait.to_csv(f"out/pheno/{trait}_duffy.tsv", sep="\t")

2022-02-28 14:34.04 [info     ] admix.Dataset: read local ancestry from /u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr/imputed/chr1.lanc
2022-02-28 14:34.22 [info     ] admix.Dataset: read local ancestry from /u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr/imputed/chr1.lanc
2022-02-28 14:34.38 [info     ] admix.Dataset: read local ancestry from /u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr/imputed/chr1.lanc
2022-02-28 14:34.54 [info     ] admix.Dataset: read local ancestry from /u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr/imputed/chr1.lanc
2022-02-28 14:35.10 [info     ] admix.Dataset: read local ancestry from /u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr/imputed/chr1.lanc
2022-02-28 14:35.27 [info     ] admix.Dataset: read local ancestry from /u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-datas